## Goals
Write a program that easily adjusts the labels for the prelabeling of the Gold Standard Dataset. This version 4 should adhere to these [guidelines](https://docs.google.com/document/d/1uaqvDKNMD2A8_FPqZlemjwAXSF2pZRoihjRnL1f1vmk/edit?usp=sharing) which includes labeling:
 - Diseases
 - Epi modifiers (not all adjectives)
 - Biological sex (not gender)
 - Dates and date ranges (for the study period)
 - Ethnicity (also includes races and nationality due to greedy categorical search)

### Epi Modifiers
For this type of entity, we will be breaking each phrase into three parts: a root, a pre-root modifier, and a post-root modifier. These phrases will be of the form “pre-root_modifier root post-root_modifier” where modifiers may or may not be present. Label these as {B or I}-EPI.

Label this list of roots in all cases except when the epi rate or disease is not valid (i.e. when it refers to a symptom of a disease:
- Incidence(s)
- prevalence(s)
- occurrence(s)  

And this list of roots, if the root refers to an epi rate:
- Affect
- Occurs
- Affects  
- Frequency
- Frequencies  

These are pre-root modifiers that I found from a corpus analysis that should be labeled:
- Annualized
- Age-adjusted
- Sex-adjusted
- Annual
- Annualized
- Associated
- Population-based
- Calculated
- Combined
- Corrected
- Cumulative
- Estimated
- Familial
- race/ethnicity-specific
- Race-specific
- Birth
- Average
- Birth
- Community-based
- Overall
- Point
- Total
- Age-specific
- Ethnicity-specific
- Overall  

These are post-root modifiers that I found from a corpus analysis:
- estimate(s)
- rate(s)

In [1]:
import csv
from nltk import tokenize
import re
from nltk.stem import PorterStemmer
porter = PorterStemmer()
print(porter.stem('Li'))

li


### Defining/importing phrases in the categories that are being labeled

In [2]:
## STOPWORDS
from nltk.corpus import stopwords
#nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))
import string
PUNCTUATION = set(char for char in string.punctuation)
STOPWORDS = STOPWORDS.union(PUNCTUATION)

## EPI PHRASES
EPI_ROOTS = {'incidence','prevalence','occurrence','incidences','prevalences','occurrences'}
EPI_CONDITIONAL_ROOTS = {'affect','occurs','affects','frequency','frequencies'} #exluding affect
EPI_PREMODIFIERS = {'annual','overall','estimated','weighted','nationwide','pooled','average','cumulative','annualized','age-adjusted','sex-adjusted','associated','population-based','calculated','combined','corrected','familial','race/ethnicity-specific','race-specific','birth','community-based','point','total','age-specific','ethnicity-specific'}
EPI_POSTMODIFIERS = {'estimate','estimates','rate','rates'}

EPI_PHRASES = set()
for root in EPI_ROOTS:
    for premod in EPI_PREMODIFIERS:
        for postmod in EPI_POSTMODIFIERS:
            EPI_PHRASES.add(' '.join([premod,root,postmod]))
            EPI_PHRASES.add(' '.join([premod,root]))
            EPI_PHRASES.add(' '.join([root,postmod]))
EPI_PHRASES = EPI_PHRASES.union(EPI_ROOTS)
            
EPI_CONDITIONAL_PHRASES = set()
for root in EPI_CONDITIONAL_ROOTS:
    for premod in EPI_PREMODIFIERS:
        for postmod in EPI_POSTMODIFIERS:
            EPI_CONDITIONAL_PHRASES.add(' '.join([premod,root,postmod]))
            EPI_CONDITIONAL_PHRASES.add(' '.join([premod,root]))
            EPI_CONDITIONAL_PHRASES.add(' '.join([root,postmod]))
EPI_CONDITIONAL_PHRASES = EPI_CONDITIONAL_PHRASES.union(EPI_CONDITIONAL_ROOTS)
            
## DATES
#ADDING every possible English date presentation to a set. easier that using spaCy and messing up the tokenization
MONTHS = {'january','february','march','april','may','june','july','august','september','october','november','december'}
YEARS = {str(i) for i in range(1900, 2022)}
DAYS= {str(i) for i in range(1, 32)}
DATES = set()
for year in YEARS:
    for month in MONTHS:
        for day in DAYS:
            DATES.add(' '.join([day,month,year]))
            DATES.add(' '.join([month,day,year]))
            DATES.add(' '.join([month,day,',',year]))
            DATES.add(' '.join([month,day+',',year]))
            DATES.add(' '.join([month,year]))
DATES = DATES.union(YEARS)

## ETHNITICIES
#from a Wikipedia Webscrape
import json
with open('ethnicities.json', 'r') as f:
    data = json.load(f)
    ETHNICITIES = {ethnicity for ethnicity in data['ethnicities']}      
    
## DISEASES
from extract_abs import load_GARD_diseases
GARD_dict, max_length = load_GARD_diseases()

## BIOLOGICAL SEX
SEX = {'male','female','males','females','girl','girls','boy','boys','man','men','woman','women','intersex','XYY','XXXY','XXXXY','klinefelter syndrome','klinefelter'}
#Klinefelter included, not including Turner syndrome because it is a rare disease

## RANGE WORDS (those are different types of dashes)
RANGE_WORDS = {'-', 'and', 'to', 'until', '‑', '‒', '–', '—', '―'}

In [ ]:
'December 31 , 2007'.lower() in DATES

### Importing files to modify tags

In [3]:
epi_train_tokens, epi_train_labels = [],[]
with open('epi_train_setV3.tsv','r', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter="\t")
    sentence_tokens,sentences_tags=[],[]
    for row in reader:
        if len(row)%2==0:
            if len(row)==0:
                if len(sentence_tokens) != len(sentences_tags):
                    print('uh oh', sentence_tokens, sentences_tags, sep='\n')
                epi_train_tokens.append(sentence_tokens.copy())
                epi_train_labels.append(sentences_tags.copy())
                sentence_tokens.clear()
                sentences_tags.clear()
            else:
                sentence_tokens.append(row[0])
                sentences_tags.append(row[1])
f.close()
print(len(epi_train_tokens),len(epi_train_labels))

4580 4580


In [4]:
epi_val_tokens, epi_val_labels= [],[]
with open('epi_val_setV3.tsv','r', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter="\t")
    sentence_tokens,sentences_tags=[],[]
    for row in reader:
        if len(row)%2==0:
            if len(row)==0:
                epi_val_tokens.append(sentence_tokens.copy())
                epi_val_labels.append(sentences_tags.copy())
                sentence_tokens.clear()
                sentences_tags.clear()
            else:
                sentence_tokens.append(row[0])
                sentences_tags.append(row[1])
f.close()
print(len(epi_val_tokens),len(epi_val_labels))

1227 1227


In [5]:
#Need to modify test to calculate labeling accuracy
epi_test_tokens, epi_test_labels= [],[]
with open('epi_test_setV3.tsv','r', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter="\t")
    sentence_tokens,sentences_tags=[],[]
    for row in reader:
        if len(row)%2==0:
            if len(row)==0:
                epi_test_tokens.append(sentence_tokens.copy())
                epi_test_labels.append(sentences_tags.copy())
                sentence_tokens.clear()
                sentences_tags.clear()
            else:
                sentence_tokens.append(row[0])
                sentences_tags.append(row[1])
f.close()
print(len(epi_test_tokens),len(epi_test_labels))

534 534


### Implement NormMap

Update: Start from `read_NormMap` since NormMap has been run

In [ ]:
import sys
sys.path.append('./NormMap')
import utils

In [ ]:
#implement dictionary of form {index:sentence} because NormMap takes JSON input
def sentences2json(tokens, name):
    dictionary = dict()
    for i,sent in enumerate(tokens):
        sentence = ' '.join(tokens[i])
        dictionary[i] = sentence
    with open('./NormMap/'+name, 'w') as f:
        json.dump(dictionary, f)
    print('SAVED: ',name)

In [ ]:
sentences2json(epi_train_tokens,'epi_train_tokens.json')
sentences2json(epi_val_tokens,'epi_val_tokens.json')
sentences2json(epi_test_tokens,'epi_test_tokens.json')

At this point run this shell command `sh NormMap_run.sh` or the constituent components

In [6]:
def read_NormMap(name):
    NormMapDict = dict()
    with open('./NormMap/'+name+'_mapped.txt','r', encoding="utf-8") as f:
        reader = csv.reader(f, delimiter="\t")
        next(reader)
        for i, row in enumerate(reader):
            if row[3]!='NA':
                NormMapDict[i-1] = row[3]
    print(name+' IMPORTED')
    return NormMapDict 

In [7]:
NormMap_train = read_NormMap('epi_train_tokens')
NormMap_val = read_NormMap('epi_val_tokens')
NormMap_test = read_NormMap('epi_test_tokens')

epi_train_tokens IMPORTED
epi_val_tokens IMPORTED
epi_test_tokens IMPORTED


In [8]:
len(NormMap_train)+len(NormMap_val)+len(NormMap_test)

1233

In [9]:
unique = {GARDID for GARDID in set(NormMap_train.values()).union(set(NormMap_val.values()),set(NormMap_test.values()))}
len(unique)

468

#### Write function that will calculate precision, recall, F1 of NormMap compared to the exact string matching
- note this is a litte harder to do based on ground truth due to the difficulty in finding all false negatives not output by both models, must be approximated based on the hits that show up and then manually checking some stuff

In [10]:
#Compares the set of GARD IDs in each set and adds fp, fn, tp, together
#Counts each assume that the other set is ground truth
def count_metrics(exact_match_sentence_set,norm_map_sentence_set):
    intersect = exact_match_sentence_set.intersection(norm_map_sentence_set)
    #exact_fp = 
    exact_fn = len(norm_map_sentence_set.difference(intersect))
    #norm_fp = 
    norm_fn = len(exact_match_sentence_set.difference(intersect))
    exact_tp = len(exact_match_sentence_set)
    norm_tp =  len(norm_map_sentence_set)
    return exact_tp,exact_fn,norm_tp,norm_fn

#### Conclusions: NormMap did not work well, abandon effort

### Doing the tagging

In [11]:
def combine_dates(sentence_tokens,sentence_labels): #this is with lists of sentence_tokens, not lists of sentences which are lists of sentence_tokens
    for i in range(1,len(sentence_labels)-1):
        #Combines 
        if 'DATE' in sentence_labels[i-1] and 'DATE' in sentence_labels[i+1] and 'DATE' not in sentence_labels[i]:
            if sentence_tokens[i] in RANGE_WORDS:
                sentence_labels[i]='I-DATE'
                sentence_labels[i+1]='I-DATE'
        
    return sentence_tokens,sentence_labels

In [12]:
# Diseases = GARD_dict.keys()
# Epi modifiers (not all adjectives) = EPI_PHRASES, EPI_CONDITIONAL_PHRASES
# Biological sex (not gender) = SEX
# Dates and date ranges (for the study period) = DATES
# Ethnicity (also includes races and nationality due to greedy categorical search) = ETHNICITIES

def tag_everything(sentence_tokens,sentence_labels):   
    i=0
    
    SentHasSTAT = bool('B-STAT' in sentence_labels)
    
    while i <len(sentence_tokens):       
        if (len(sentence_tokens)-i) < max_length:
            compare_length=len(sentence_tokens)-i
        else:
            compare_length = max_length
        #Compares longest sequences first and goes down until there is a match
        exit = False
        while compare_length>0:
            s = ' '.join(sentence_tokens[i:i+compare_length])
            
            #DISEASES
            if s.lower() in GARD_dict.keys():
                print('DISEASE: ',s)
                sentence_labels[i] = 'B-DIS'
                for j in range(i+1,i+compare_length):
                    sentence_labels[j] = 'I-DIS'
                i+=compare_length-1
                break
            
            #EPI TYPE PHRASES
            elif s.lower() in EPI_PHRASES or (s.lower() in EPI_CONDITIONAL_PHRASES and SentHasSTAT):
                #print('EPI TYPE: ',s)
                sentence_labels[i] = 'B-EPI'
                for j in range(i+1,i+compare_length):
                    sentence_labels[j] = 'I-EPI'
                i+=compare_length-1
                break
            
            #SEX
            elif s.lower() in SEX:
                #print('SEX: ',s)
                sentence_labels[i] = 'B-SEX'
                for j in range(i+1,i+compare_length):
                    sentence_labels[j] = 'I-SEX'
                i+=compare_length-1
                break
            
            #DATES
            elif s.lower() in DATES:
                #print('DATE: ',s)
                sentence_labels[i] = 'B-DATE'
                for j in range(i+1,i+compare_length):
                    sentence_labels[j] = 'I-DATE'
                i+=compare_length-1
                break
            
            #ETHNICITIES
            elif s in ETHNICITIES:
                #print('RAW ETHNICITY: ',s)
                sentence_labels[i] = 'B-ETHN'
                for j in range(i+1,i+compare_length):
                    sentence_labels[j] = 'I-ETHN'
                i+=compare_length-1
                break
            
            #stem every word in the phrase and see if it matches
            elif ' '.join([porter.stem(word) for word in s.split()]) in ETHNICITIES:
                #print('STEMMED ETHNICITY: ',s)
                sentence_labels[i] = 'B-ETHN'
                for j in range(i+1,i+compare_length):
                    sentence_labels[j] = 'I-ETHN'
                i+=compare_length-1
                break
                
            else:
                compare_length-=1
        i+=1  
    return sentence_tokens, sentence_labels

In [ ]:
from extract_abs import autosearch
def modify_labels(tokens,labels,NormMap_dict):
    if len(tokens)!=len(labels):
        raise IndexError("Num Sentences {} and Num Sentence Labels {} Mismatch".format(len(tokens),len(labels)))
    
    #exact_tp,exact_fn,norm_tp,norm_fn = 0,0,0,0
    
    for i in range(len(tokens)):
        if len(tokens[i])!=len(labels[i]):
            raise IndexError("Sentence Length {} and Label Length {} Mismatch".format(len(tokens[i]),len(labels[i])))
            
        tokens[i], labels[i] = tag_everything(tokens[i], labels[i])
        tokens[i], labels[i] = combine_dates(tokens[i], labels[i])
        
        if i in NormMap_dict.keys():
            print('NormMap sentence: ',tokens[i])
            #print all related GARD names:
            print('NormMap GARD ID: ',NormMap_dict[i])
            autosearch(NormMap_dict[i],GARD_dict)
            #print('Autosearch',)
            print('')
        
        if i %250==0:
            print(i)
    
    return tokens, labels#, {'exact_tp':exact_tp,'exact_fn':exact_fn,'norm_tp':norm_tp,'norm_fn':norm_fn}

In [ ]:
mod_train_tokens, mod_train_labels = modify_labels(epi_train_tokens,epi_train_labels,NormMap_train)

In [ ]:
mod_val_tokens, mod_val_labels = modify_labels(epi_val_tokens,epi_val_labels,NormMap_val)

In [ ]:
mod_test_tokens, mod_test_labels = modify_labels(epi_test_tokens,epi_test_labels,NormMap_test)

With NormMap Metrics

In [13]:
from extract_abs import autosearch
def modify_labels(tokens,labels,NormMap_dict):
    if len(tokens)!=len(labels):
        raise IndexError("Num Sentences {} and Num Sentence Labels {} Mismatch".format(len(tokens),len(labels)))
    
    exact_tp,exact_fn,norm_tp,norm_fn = 0,0,0,0
    
    for i in range(len(tokens)):
        if len(tokens[i])!=len(labels[i]):
            raise IndexError("Sentence Length {} and Label Length {} Mismatch".format(len(tokens[i]),len(labels[i])))
            
        tokens[i], labels[i] = tag_everything(tokens[i], labels[i])
        tokens[i], labels[i] = combine_dates(tokens[i], labels[i])
        
        #This was the quantitative comparison between the exact string match and NormMap but it does not work well enough to do it systematically.

        #Print out both results
        if i in NormMap_dict.keys() or 'B-DIS' in labels[i]:
            if i in NormMap_dict.keys():
                print('NormMap sentence: ',tokens[i])
                #print all related GARD names:
                print('NormMap GARD ID: ',NormMap_dict[i])
                autosearch(NormMap_dict[i],GARD_dict)
                norm_map_sentence_set = set([NormMap_dict[i]])
            else:
                norm_map_sentence_set = set()
            
            if 'B-DIS' in labels[i]:
                print('Exact Match sentence: ',tokens[i])
                dz_indices = [j for j in range(len(labels[i])) if labels[i][j] in {'B-DIS','I-DIS'}]
                exact_match_sentence_set = set()
                disease = []
                print(dz_indices)
                for cnt, val in enumerate(dz_indices):
                    #case for list of length 1
                    if len(dz_indices) ==1:
                        term = tokens[i][val].lower()
                        if term in GARD_dict.keys(): 
                            GARD_ID = GARD_dict[term]
                            print('Exact Match GARD ID: ',GARD_ID)
                            autosearch(GARD_ID,GARD_dict)
                            #print([name for name,ID in GARD_dict.items() if ID == GARD_ID])
                            exact_match_sentence_set.add(GARD_ID)
                    #case for what to do when at last position in list
                    elif cnt==len(dz_indices)-1:
                        disease.append(tokens[i][val])
                        term = ' '.join(disease).lower()
                        if term in GARD_dict.keys(): 
                            GARD_ID = GARD_dict[term]
                            print('Exact Match GARD ID: ',GARD_ID)
                            autosearch(GARD_ID,GARD_dict)
                            #print([name for name,ID in GARD_dict.items() if ID == GARD_ID])
                            exact_match_sentence_set.add(GARD_ID)
                        else:
                            print(term,"ERROR 1",disease)
                    #case for what to do when there are multiple diseases in single sentence
                    #if len(disease)>1 and (dz_indices[cnt+1]-val)>1: #if there was jumps, trying to find the disease word chunks
                    elif cnt>0 and len(disease)>0 and val-dz_indices[cnt-1]>1: #if there was jumps, trying to find the disease word chunks
                        term = ' '.join(disease).lower()
                        if term in GARD_dict.keys(): 
                            GARD_ID = GARD_dict[term]
                            print('Exact Match GARD ID: ',GARD_ID)
                            autosearch(GARD_ID,GARD_dict)
                            #print([name for name,ID in GARD_dict.items() if ID == GARD_ID])
                            exact_match_sentence_set.add(GARD_ID)
                        else:
                            print(term,"ERROR 2",disease)
                        disease.clear()
                        disease.append(tokens[i][val])
                        if tokens[i][val].lower() in GARD_dict.keys(): 
                            GARD_ID = GARD_dict[tokens[i][val].lower()]
                            print('Exact Match GARD ID: ',GARD_ID)
                            autosearch(GARD_ID,GARD_dict)
                            #print([name for name,ID in GARD_dict.items() if ID == GARD_ID])
                            exact_match_sentence_set.add(GARD_ID)
                    #Otherwise just add to disease list
                    else:
                        #print('adding',tokens[i][val],'to the list')
                        disease.append(tokens[i][val])
            else:
                exact_match_sentence_set = set()
            
            ex_tp,ex_fn,nm_tp,nm_fn = count_metrics(exact_match_sentence_set,norm_map_sentence_set)
            exact_tp+=ex_tp
            exact_fn+=ex_fn
            norm_tp+=nm_tp
            norm_fn+=nm_fn
            print('')
        #if i %250==0:
        #    print(i)
    
    return tokens, labels, {'exact_tp':exact_tp,'exact_fn':exact_fn,'norm_tp':norm_tp,'norm_fn':norm_fn}

In [14]:
mod_train_tokens, mod_train_labels, train_results = modify_labels(epi_train_tokens,epi_train_labels,NormMap_train)

NormMap sentence:  ['Dyslipidemia', 'and', 'baseline', 'ejection', 'fraction', 'were', 'seen', 'as', 'the', 'major', 'risk', 'factors', 'associated', 'with', 'the', 'higher', 'incidence', 'of', 'CICM', '.']
NormMap GARD ID:  GARD:0010468
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['cardiac rupture']

DISEASE:  Cardiac rupture
NormMap sentence:  ['Background', 'Cardiac', 'rupture', 'is', 'a', 'disastrous', 'but', 'uncommon', 'complication', 'of', 'acute', 'ST', '-', 'elevation', 'myocardial', 'infarction', '(', 'STEMI', ')', '.']
NormMap GARD ID:  GARD:0010468
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['cardiac rupture']
Exact Match sentence:  ['Background', 'Cardiac', 'rupture', 'is', 'a', 'disastrous', 'but', 'uncommon', 'complication', 'of', 'acute', 'ST', '-', 'elevation', 'myocardial', 'infarction', '(', 'STEMI', ')', '.']
[1, 2]
Exact Match GARD ID:  GARD:0010468
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['cardiac rupture']

DISEASE:  cardi

DISEASE:  Albright hereditary osteodystrophy
NormMap sentence:  ['Heterozygous', 'inactivating', 'mutations', 'involving', 'the', 'maternal', 'GNAS', 'exons', '1', '-', '13', 'that', 'encodes', 'the', 'alpha', 'subunit', 'of', 'the', 'stimulatory', 'G', 'protein', '(', 'Gsα', ')', 'cause', 'inactivating', 'parathyroid', 'hormone', '(', 'PTH)/PTHrP', 'signalling', 'disorder', 'type', '2', '(', 'iPPSD2', 'or', 'PHP', 'type', '1A', ')', ',', 'which', 'is', 'characterized', 'by', 'Albright', 'hereditary', 'osteodystrophy', 'and', 'resistance', 'to', 'multiple', 'hormones', 'that', 'act', 'through', 'the', 'Gsα', 'signalling', 'pathway', '(', 'including', 'PTH', ',', 'thyroid', '-', 'stimulating', 'hormone', ',', 'and', 'α', '-', 'melanocyte', '-', 'stimulating', 'hormone', ')', '.']
NormMap GARD ID:  GARD:0000193
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['opitz g/bbb syndrome', 'hypospadias-dysphagia, syndrome', 'opitz-frias syndrome', 'g syndrome', 'opitz-g syndrome, type 2

NormMap sentence:  ['Aim', ':', 'Sentinel', 'lymph', 'node', 'biopsy', '(', 'SLNB', ')', 'is', 'the', 'accepted', 'approach', 'to', 'stage', 'the', 'clinically', 'negative', 'axilla', '.']
NormMap GARD ID:  GARD:0007383
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['phenylketonuria', 'phenylalanine hydroxylase deficiency', 'oligophrenia phenylpyruvica', 'folling disease']

DISEASE:  Q fever
Exact Match sentence:  ['Background', 'In', 'South', 'Korea', ',', 'the', 'number', 'of', 'Q', 'fever', 'cases', 'has', 'rapidly', 'increased', 'since', '2015', '.']
[8, 9]
Exact Match GARD ID:  GARD:0007515
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['q fever', 'q fever pneumonia', 'coxiella burnetii fever', 'query fever']

DISEASE:  Q fever
Exact Match sentence:  ['Therefore', ',', 'this', 'study', 'aimed', 'to', 'characterize', 'the', 'epidemiological', 'and', 'clinical', 'features', 'of', 'Q', 'fever', 'in', 'South', 'Korea', 'between', '2011', 'and', '2017', '.']
[13, 14

NormMap sentence:  ['Given', 'that', 'the', 'dyslipidemia', '-', 'related', 'autoimmune', 'diseases', 'are', 'caused', 'by', 'autoreactive', 'T', 'cells', 'and', 'B', 'cells', ',', 'dyslipidemia', 'seems', 'to', 'directly', 'or', 'indirectly', 'regulate', 'the', 'adaptive', 'immunity', '.']
NormMap GARD ID:  GARD:0007515
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['q fever', 'q fever pneumonia', 'coxiella burnetii fever', 'query fever']

NormMap sentence:  ['Indeed', ',', 'accumulating', 'evidence', 'has', 'unveiled', 'that', 'proatherogenic', 'factors', 'can', 'impact', 'the', 'differentiation', 'and', 'function', 'of', 'CD4', '+', 'T', 'cells', ',', 'CD8', '+', 'T', 'cells', ',', 'and', 'B', 'cells', '.']
NormMap GARD ID:  GARD:0007515
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['q fever', 'q fever pneumonia', 'coxiella burnetii fever', 'query fever']

DISEASE:  osteomyelitis
Exact Match sentence:  ['Background', 'Tuberculosis', 'osteomyelitis', 'is', 'rarel

NormMap sentence:  ['A', 'maternal', 'interview', ',', 'conducted', 'within', '6', 'months', 'of', 'delivery', ',', 'collected', 'information', 'on', 'demographics', ',', 'reproductive', 'history', ',', 'diet', ',', 'and', 'supplement', 'use', '.']
NormMap GARD ID:  GARD:0005194
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['idiopathic thrombocytopenic purpura', 'autoimmune thrombocytopenic purpura', 'thrombocytopenic purpura autoimmune']

NormMap sentence:  ['Daily', 'folic', 'acid', 'intake', 'in', 'the', 'periconceptional', 'period', 'was', 'calculated', 'using', 'both', 'dietary', 'and', 'supplement', 'information', 'and', 'categorized', 'as', 'low', 'intake', '(', '<', '400', 'µg', '/', 'day', ')', 'or', 'high', 'intake', '(', '≥400', 'µg', '/', 'day', ')', '.']
NormMap GARD ID:  GARD:0005870
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['autoimmune hemolytic anemia', 'anemia hemolytic autoimmune', 'acquired autoimmune hemolytic anemia', 'idiopathic autoimmun

...

NormMap sentence:  ['The', 'etiology', 'is', 'multifactorial', 'and', 'requires', 'a', 'judicious', 'assessment', 'of', 'each', 'clinical', 'scenario', '.']
NormMap GARD ID:  GARD:0000916
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['blount disease', 'tibia vara', 'osteochondrosis deformans tibiae', "blount's disease", 'blount-barber syndrome', 'erlacher-blount syndrome', 'osteochondrosis deformans tibiae, familial infantile type']

NormMap sentence:  ['The', 'diagnosis', 'and', 'its', 'management', 'are', 'further', 'complicated', 'as', 'most', 'neonatal', 'seizures', 'may', 'have', 'very', 'subtle', 'or', 'no', 'clinical', 'changes', 'and', 'the', 'diagnosis', 'may', 'be', 'just', 'based', 'on', 'EEG', 'findings', ',', 'so', '-', 'called', 'subclinical', '.']
NormMap GARD ID:  GARD:0012521
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['multifocal fibrosclerosis', 'igg4-related disease', 'igg4-related systemic disease', 'igg4-syndrome', 'igg4-associated disease'

DISEASE:  scurvy
Exact Match sentence:  ['Roncalli', 'Parolino', 'focused', 'his', 'study', 'on', 'the', 'treatment', 'of', 'scurvy', 'and', 'he', 'promoted', 'the', 'inclusion', 'of', 'the', 'Greenlandic', 'and', 'Faroese', 'therapy', 'into', 'the', 'broader', 'European', 'context', '.']
[9]
Exact Match GARD ID:  GARD:0010406
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['scurvy', 'vitamin c deficiency', 'deficiency of vitamin c', 'scorbutus', 'hypoascorbemia']

DISEASE:  VATER association
DISEASE:  anal atresia
Exact Match sentence:  ['VACTERL', '/', 'VATER', 'association', 'is', 'typically', 'defined', 'by', 'the', 'presence', 'of', 'at', 'least', 'three', 'of', 'the', 'following', 'congenital', 'malformations', ':', 'vertebral', 'defects', ',', 'anal', 'atresia', ',', 'cardiac', 'defects', ',', 'tracheo', '-', 'esophageal', 'fistula', ',', 'renal', 'anomalies', ',', 'and', 'limb', 'abnormalities', '.']
[2, 3, 23, 24]
Exact Match GARD ID:  GARD:0005443
SEARCH TERM MATCHED

NormMap sentence:  ['Motility', 'testing', 'was', 'performed', 'and', 'reported', 'in', '42', '(', '19.2', '%', ')', 'patients', '.']
NormMap GARD ID:  GARD:0007173
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['neonatal herpes', 'neonatal herpes simplex virus infection', 'neonatal hsv infection']

DISEASE:  gastroparesis
Exact Match sentence:  ['Out', 'of', 'them', ',', 'five', '(', '11.9', '%', ')', 'had', 'esophageal', 'dysmotility', ',', '18', '(', '42.8', '%', ')', 'had', 'gastroparesis', ',', 'five', '(', '11.9', '%', ')', 'had', 'small', 'bowel', '/', 'colon', 'altered', 'transit', 'time', ',', 'and', 'four', '(', '9.5', '%', ')', 'had', 'global', 'dysmotility', '.']
[19]
Exact Match GARD ID:  GARD:0012278
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['gastroparesis', 'delayed gastric emptying']

DISEASE:  Arrhythmogenic right ventricular cardiomyopathy
Exact Match sentence:  ['The', 'KEGG', 'analysis', 'revealed', 'that', 'the', 'DEPs', 'showed', 'signific

NormMap sentence:  ['The', 'presence', 'of', 'both', 'ECG', '-', 'LVH', 'and', 'renal', 'damage', 'was', 'associated', 'with', 'the', 'higher', 'prevalence', '(', 'OR', '3.12', ',', '[', '2.33', '-', '4.19', ']', ')', '.']
NormMap GARD ID:  GARD:0004410
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['polydactyly', 'extra digits', 'supernumerary digits', 'polydactylia', 'hyperdactyly', 'polydactylism', 'non-syndromic polydactyly']

NormMap sentence:  ['Conclusion', 'In', 'patients', 'with', 'HT', 'and', 'type', '2', 'DM', ',', 'ECG', '-', 'LVH', 'or', 'renal', 'damage', ',', 'evaluated', 'using', 'simple', 'methods', ',', 'are', 'associated', 'with', 'an', 'increased', 'prevalence', 'of', 'established', 'CVD', '.']
NormMap GARD ID:  GARD:0004410
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['polydactyly', 'extra digits', 'supernumerary digits', 'polydactylia', 'hyperdactyly', 'polydactylism', 'non-syndromic polydactyly']

DISEASE:  Fanconi anemia
Exact Match sentenc

NormMap sentence:  ['The', 'most', 'common', 'functional', 'signs', 'were', 'weight', 'loss', ',', 'tremors', 'and', 'palpitations', '.']
NormMap GARD ID:  GARD:0000467
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['propionic acidemia', 'propionyl-coa carboxylase deficiency', 'pcc deficiency', 'glycinemia, ketotic', 'hyperglycinemia with ketoacidosis and leukopenia', 'ketotic hyperglycinemia', 'ketotic glycinemia', 'propionicacidemia']

NormMap sentence:  ['Mean', 'FT4', 'was', '54,51', '±', '19,56', 'ng', '/', 'dl', '(', 'ranging', 'from', '8,90', 'and', '100', ')', '.']
NormMap GARD ID:  GARD:0007033
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['methylmalonic acidemia', 'acidemia, methylmalonic']

NormMap sentence:  ['Huang', 'et', 'al', '.']
NormMap GARD ID:  GARD:0012521
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['multifocal fibrosclerosis', 'igg4-related disease', 'igg4-related systemic disease', 'igg4-syndrome', 'igg4-associated disease', 'igg

In [15]:
mod_val_tokens, mod_val_labels, val_results = modify_labels(epi_val_tokens,epi_val_labels,NormMap_val)

DISEASE:  amyotrophic lateral sclerosis
Exact Match sentence:  ['Most', 'amyotrophic', 'lateral', 'sclerosis', '(', 'ALS', ')', 'cases', 'are', 'considered', 'sporadic', ',', 'without', 'a', 'known', 'genetic', 'basis', ',', 'and', 'lifestyle', 'factors', 'are', 'suspected', 'to', 'play', 'an', 'etiologic', 'role', '.']
[1, 2, 3]
Exact Match GARD ID:  GARD:0005786
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['amyotrophic lateral sclerosis', 'lou gehrig disease', 'amyotrophic lateral sclerosis type 1', 'charcot disease', 'motor neurone disease']

NormMap sentence:  ['This', 'limits', 'our', 'understanding', 'of', 'the', 'epidemiology', 'of', 'the', 'disease', 'especially', 'for', 'planning', 'and', 'implementing', 'quantifiable', 'and', 'cost', '-', 'effective', 'control', 'measures', '.']
NormMap GARD ID:  GARD:0000183
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['bubonic plague']

DISEASE:  bubonic plague
Exact Match sentence:  ['To', 'fill', 'this', 'gap', ','

DISEASE:  HTLV-1
Exact Match sentence:  ['A', 'molecular', 'epidemiological', 'study', 'was', 'conducted', 'in', 'a', 'population', 'of', '9422', 'blood', 'donors', 'in', 'the', 'province', 'of', 'Corrientes', ',', 'Northeastern', 'Argentina', ',', 'to', 'determine', 'the', 'prevalence', 'of', 'Human', 'T', '-', 'cell', 'lymphotropic', 'virus', 'types', '1', 'and', '2', '(', 'HTLV-1/2', ')', ',', 'the', 'phylogenetic', 'identification', 'of', 'HTLV-1', 'and', '2', 'subtypes', '/', 'subgroups', 'and', 'perform', 'a', 'mutation', 'analysis', '.']
[45]
Exact Match GARD ID:  GARD:0009645
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['human t-cell leukemia virus type 1', 'htlv-1', 'human t lymphotropic virus type 1']

DISEASE:  HTLV-1
DISEASE:  HTLV-2
NormMap sentence:  ['Based', 'on', 'the', 'results', 'obtained', ',', 'it', 'was', 'shown', 'that', 'both', 'HTLV-1', 'and', 'HTLV-2', 'are', 'circulating', 'in', 'a', 'low', '-', 'risk', 'population', 'of', 'Corrientes', ',', 'alth

DISEASE:  COL2A1
NormMap sentence:  ['Results', 'Fifty', '-', 'percent', 'of', 'patients', 'with', 'type', 'I', '(', 'COL2A1', ')', 'required', 'MDO', 'as', 'a', 'neonate', 'compared', 'to', 'only', '31', '%', 'of', 'patients', 'with', 'type', 'II', '(', 'COL11A1', ')', ',', 'though', 'the', 'difference', 'between', 'the', 'two', 'groups', 'was', 'not', 'statistically', 'significant', '.']
NormMap GARD ID:  GARD:0010782
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['stickler syndrome']
Exact Match sentence:  ['Results', 'Fifty', '-', 'percent', 'of', 'patients', 'with', 'type', 'I', '(', 'COL2A1', ')', 'required', 'MDO', 'as', 'a', 'neonate', 'compared', 'to', 'only', '31', '%', 'of', 'patients', 'with', 'type', 'II', '(', 'COL11A1', ')', ',', 'though', 'the', 'difference', 'between', 'the', 'two', 'groups', 'was', 'not', 'statistically', 'significant', '.']
[10]
Exact Match GARD ID:  GARD:0009246
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['collagenopathy type 

DISEASE:  Alport syndrome
Exact Match sentence:  ['COL4A5', 'mutations', 'are', 'associated', 'with', 'X', '-', 'linked', 'Alport', 'syndrome', ',', 'which', 'represents', '80', 'to', '85', '%', 'of', 'cases', 'and', 'is', 'more', 'severe', 'in', 'boys', 'than', 'in', 'girls', '.']
[8, 9]
Exact Match GARD ID:  GARD:0005785
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['alport syndrome', 'alport syndrome, x-linked', 'hemorrhagic familial nephritis', 'hemorrhagic hereditary nephritis', 'congenital hereditary hematuria']

DISEASE:  Alport syndrome
Exact Match sentence:  ['Mutations', 'in', 'COL4A3', 'or', 'COL4A4', 'are', 'associated', 'with', 'autosomal', 'Alport', 'syndrome', '.']
[9, 10]
Exact Match GARD ID:  GARD:0005785
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['alport syndrome', 'alport syndrome, x-linked', 'hemorrhagic familial nephritis', 'hemorrhagic hereditary nephritis', 'congenital hereditary hematuria']

NormMap sentence:  ['In', 'Bangladesh', ',', '

NormMap sentence:  ['The', 'aim', 'of', 'this', 'section', 'is', 'to', 'comprehensively', 'report', 'on', 'the', 'epidemiology', 'of', 'key', 'infectious', 'disease', 'in', 'otorhinolaryngology', ',', 'reporting', 'on', 'their', 'burden', 'at', 'the', 'national', 'and', 'international', 'level', ',', 'expanding', 'of', 'the', 'need', 'of', 'promoting', 'and', 'implementing', 'preventive', 'interventions', ',', 'and', 'the', 'rationale', 'of', 'applying', 'evidence', '-', 'based', ',', 'effective', 'and', 'cost-', 'effective', 'diagnostic', ',', 'curative', 'and', 'preventive', 'approaches', '.']
NormMap GARD ID:  GARD:0008403
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['supranuclear ocular palsy', 'conjugate gaze palsy', 'gaze palsy']

NormMap sentence:  ['In', 'watermelon', ',', '34', 'carbohydrate', 'linkages', 'were', 'identified', 'with', 'galactose', ',', 'glucose', ',', 'and', 'arabinose', 'linkages', 'in', 'highest', 'abundance', '.']
NormMap GARD ID:  GARD:0006520


...

NormMap sentence:  ['The', 'calculated', 'prevalence', 'of', 'WD', 'was', 'comparatively', 'higher', 'than', 'that', 'of', 'previous', 'reports', ',', 'indicating', 'previous', 'underdiagnosis', 'or', 'the', 'existence', 'of', 'less', 'severe', 'phenotypes', '.']
NormMap GARD ID:  GARD:0012278
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['gastroparesis', 'delayed gastric emptying']

DISEASE:  Q fever
Exact Match sentence:  ['Purpose', 'of', 'review', 'In', 'this', 'review', ',', 'we', 'report', 'on', 'the', 'state', 'of', 'knowledge', 'about', 'human', 'Q', 'fever', 'in', 'Brazil', 'and', 'on', 'the', 'Guiana', 'Shield', ',', 'an', 'Amazonian', 'region', 'located', 'in', 'northeastern', 'South', 'America', '.']
[16, 17]
Exact Match GARD ID:  GARD:0007515
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['q fever', 'q fever pneumonia', 'coxiella burnetii fever', 'query fever']

DISEASE:  Q fever
NormMap sentence:  ['Summary', 'Q', 'fever', 'is', 'a', 'bacterial', 'zoo

DISEASE:  esophageal atresia
Exact Match sentence:  ['Results', 'The', 'DATE', 'association', 'consisted', 'of', 'type', 'C', 'esophageal', 'atresia', '(', '13', ')', ',', 'complete', '(', '9', ')', 'or', 'incomplete', '(', '4', ')', 'congenital', 'duodenal', 'obstruction', '(', 'CDO', ')', ',', 'and', 'high', 'or', 'intermediate', '(', '8)', 'or', 'low', '(', '5', ')', 'ARM', '.']
[8, 9]
Exact Match GARD ID:  GARD:0006381
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['esophageal atresia']

DISEASE:  VACTERL association
Exact Match sentence:  ['Eight', 'patients', 'had', 'at', 'least', 'one', 'additional', 'component', 'feature', 'of', 'VACTERL', 'association', '.']
[10, 11]
Exact Match GARD ID:  GARD:0005443
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['vacterl association', 'vater association']

DISEASE:  VACTERL association
Exact Match sentence:  ['Conclusions', 'The', 'DATE', 'association', 'is', 'a', 'low', '-', 'frequency', 'entity', ',', 'often', 'occurrin

NormMap sentence:  ['The', 'hierarchical', 'information', 'flow', 'through', 'DNA', '-', 'RNA', '-', 'protein', '-', 'metabolite', 'collectively', 'referred', 'to', 'as', "'", 'molecular', 'fingerprint', "'", 'defines', 'both', 'health', 'and', 'disease', '.']
NormMap GARD ID:  GARD:0000054
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['duodenal atresia', 'duodenal stenosis']

NormMap sentence:  ['Environment', 'and', 'food', '(', 'quality', 'and', 'quantity', ')', 'are', 'the', 'key', 'factors', 'known', 'to', 'affect', 'the', 'health', 'of', 'an', 'individual', '.']
NormMap GARD ID:  GARD:0006381
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['esophageal atresia']

NormMap sentence:  ['The', 'fundamental', 'concepts', 'are', 'that', 'the', 'transition', 'from', 'a', 'healthy', 'condition', 'to', 'a', 'disease', 'phenotype', 'must', 'occur', 'by', 'concurrent', 'alterations', 'in', 'the', 'genome', 'expression', 'or', 'by', 'differences', 'in', 'protein', 'synthes

NormMap sentence:  ['Maternal', 'knowledge', 'translation', 'and', 'an', 'informed', 'consent', 'process', 'with', 'risk', '-', 'benefit', 'maternal', '/', 'fetal', 'risk', 'counseling', 'are', 'required', 'prior', 'to', 'any', 'diagnostic', 'amniocentesis', 'procedure', '.']
NormMap GARD ID:  GARD:0010406
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['scurvy', 'vitamin c deficiency', 'deficiency of vitamin c', 'scorbutus', 'hypoascorbemia']

DISEASE:  Pilomatricoma
Exact Match sentence:  ['Pilomatricoma', '(', '28.2', '%', ')', 'was', 'the', 'most', 'common', 'benign', 'tumor', 'while', 'sebaceous', 'carcinoma', '(', '11.8', '%', ')', 'was', 'the', 'most', 'common', 'malignant', 'tumor', '.']
[0]
Exact Match GARD ID:  GARD:0009452
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['pilomatrixoma', 'pilomatricoma', 'calcifying epithelioma of malherbe']

NormMap sentence:  ['Methods', 'We', 'consecutively', 'enrolled', '65', 'individuals', 'referred', 'for', 'short', 's

DISEASE:  Bullous pemphigoid
Exact Match sentence:  ['Bullous', 'pemphigoid', '(', 'BP', ')', 'is', 'the', 'most', 'prevalent', 'autoimmune', 'blistering', 'skin', 'disease', 'in', 'the', 'Western', 'world', 'affecting', 'mainly', 'the', 'elderly', 'population', '.']
[0, 1]
Exact Match GARD ID:  GARD:0005972
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['bullous pemphigoid', 'senile dermatitis herpetiformis', 'pemphigoid', 'parapemphigus', 'old age pemphigus', 'benign pemphigus']

DISEASE:  dermatitis herpetiformis
Exact Match sentence:  ['Other', 'reported', 'findings', 'include', 'urticarial', 'lesions', ',', 'prurigo', '-', 'like', 'nodules', ',', 'multiple', 'small', 'vesicles', 'resembling', 'dermatitis', 'herpetiformis', 'or', 'pompholyx', ',', 'vegetating', 'and', 'purulent', 'lesions', 'localized', 'in', 'intertriginous', 'areas', ',', 'and', 'even', 'exfoliative', 'erythroderma', '.']
[16, 17]
Exact Match GARD ID:  GARD:0001917
SEARCH TERM MATCHED TO GARD DICTIONARY

In [16]:
mod_test_tokens, mod_test_labels, test_results = modify_labels(epi_test_tokens,epi_test_labels,NormMap_test)

DISEASE:  pseudohypoparathyroidism type 1b
DISEASE:  PHP1b
NormMap sentence:  ['Context', 'Patients', 'with', 'pseudohypoparathyroidism', 'type', '1b', '(', 'PHP1b', ')', 'show', 'disordered', 'imprinting', 'of', 'the', 'maternal', 'GNAS', 'allele', 'or', 'paternal', 'uniparental', 'disomy', '(', 'UPD', ')', '.']
NormMap GARD ID:  GARD:0010758
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['pseudohypoparathyroidism']
Exact Match sentence:  ['Context', 'Patients', 'with', 'pseudohypoparathyroidism', 'type', '1b', '(', 'PHP1b', ')', 'show', 'disordered', 'imprinting', 'of', 'the', 'maternal', 'GNAS', 'allele', 'or', 'paternal', 'uniparental', 'disomy', '(', 'UPD', ')', '.']
[3, 4, 5, 7]
pseudohypoparathyroidism type 1b php1b ERROR 1 ['pseudohypoparathyroidism', 'type', '1b', 'PHP1b']

DISEASE:  PHP1b
Exact Match sentence:  ['Objective', 'Characterization', 'of', 'epigenetic', 'and', 'genetic', 'defects', 'in', 'patients', 'with', 'PHP1b', '.']
[10]
Exact Match GARD ID:  GARD:00

DISEASE:  HTLV-1
Exact Match sentence:  ['The', 'prevalence', 'of', 'HTLV-1', 'in', 'the', 'Peruvian', 'Amazon', 'basin', 'is', 'about', '1.7', '%', ',', 'indicating', 'an', 'endemic', 'presence', '.']
[3]
Exact Match GARD ID:  GARD:0009645
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['human t-cell leukemia virus type 1', 'htlv-1', 'human t lymphotropic virus type 1']

DISEASE:  HTLV-1
Exact Match sentence:  ['Screening', 'for', 'HTLV-1', 'in', 'prenatal', 'care', 'is', 'warranted', '.']
[2]
Exact Match GARD ID:  GARD:0009645
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['human t-cell leukemia virus type 1', 'htlv-1', 'human t lymphotropic virus type 1']

DISEASE:  neuromyelitis optica spectrum disorder
Exact Match sentence:  ['Background', 'Comorbidity', 'may', 'influence', 'clinical', 'aspects', 'of', 'neuromyelitis', 'optica', 'spectrum', 'disorder', '(', 'NMOSD', ')', '.']
[7, 8, 9, 10]
Exact Match GARD ID:  GARD:0006267
SEARCH TERM MATCHED TO GARD DICTIONARY

DISEASE:  HTLV-1
NormMap sentence:  ['After', 'adjustment', 'for', 'age', ',', 'sex', ',', 'and', 'hepatitis', 'C', 'virus', 'infection', ',', 'HTLV-1', 'was', 'significantly', 'associated', 'with', 'prevalent', 'RA', '(', 'odds', 'ratio', ',', '2.89', ';', '95', '%', 'confidence', 'interval', ',', '1.06', ',', '7.03', ')', '.']
NormMap GARD ID:  GARD:0009645
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['human t-cell leukemia virus type 1', 'htlv-1', 'human t lymphotropic virus type 1']
Exact Match sentence:  ['After', 'adjustment', 'for', 'age', ',', 'sex', ',', 'and', 'hepatitis', 'C', 'virus', 'infection', ',', 'HTLV-1', 'was', 'significantly', 'associated', 'with', 'prevalent', 'RA', '(', 'odds', 'ratio', ',', '2.89', ';', '95', '%', 'confidence', 'interval', ',', '1.06', ',', '7.03', ')', '.']
[13]
Exact Match GARD ID:  GARD:0009645
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['human t-cell leukemia virus type 1', 'htlv-1', 'human t lymphotropic virus type 

DISEASE:  granuloma annulare
Exact Match sentence:  ['Objectives', 'To', 'estimate', 'the', 'population', '-', 'based', 'incidence', 'and', 'prevalence', 'of', 'granuloma', 'annulare', 'in', 'the', 'United', 'States', 'and', 'to', 'identify', 'the', 'most', 'commonly', 'prescribed', 'treatments', '.']
[11, 12]
Exact Match GARD ID:  GARD:0006546
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['granuloma annulare']

DISEASE:  granuloma annulare
NormMap sentence:  ['Design', ',', 'setting', ',', 'and', 'participants', 'This', 'cross', '-', 'sectional', 'study', 'used', 'deidentified', 'data', 'from', 'the', 'Optum', 'Clinformatics', 'Data', 'Mart', 'Database', 'from', 'January', '1', ',', '2017', ',', 'to', 'December', '31', ',', '2018', ',', 'to', 'identify', 'patients', 'with', 'granuloma', 'annulare', '.']
NormMap GARD ID:  GARD:0000550
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['adenylosuccinase deficiency', 'adenylosuccinate lyase deficiency', 'adsl deficiency'

NormMap sentence:  ['Conclusions', 'and', 'relevance', 'Granuloma', 'annulare', 'is', 'a', 'rare', 'disease', 'in', 'the', 'United', 'States', 'that', 'is', 'more', 'common', 'among', 'women', 'and', 'middle', '-', 'aged', 'to', 'older', 'individuals', '.']
NormMap GARD ID:  GARD:0006546
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['granuloma annulare']
Exact Match sentence:  ['Conclusions', 'and', 'relevance', 'Granuloma', 'annulare', 'is', 'a', 'rare', 'disease', 'in', 'the', 'United', 'States', 'that', 'is', 'more', 'common', 'among', 'women', 'and', 'middle', '-', 'aged', 'to', 'older', 'individuals', '.']
[3, 4]
Exact Match GARD ID:  GARD:0006546
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['granuloma annulare']

DISEASE:  granuloma annulare
Exact Match sentence:  ['The', 'findings', 'of', 'this', 'cross', '-', 'sectional', 'study', 'provide', 'important', 'background', 'regarding', 'the', 'basic', 'epidemiology', 'and', 'overall', 'burden', 'of', 'granulom

NormMap sentence:  ['Conclusion', 'In', 'front', 'of', 'a', 'floppy', 'infant', ',', 'a', 'large', 'variety', 'of', 'disorders', 'should', 'be', 'considered', ',', 'including', 'some', 'connective', 'diseases', '.']
NormMap GARD ID:  GARD:0005575
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['prader-willi syndrome', 'willi-prader syndrome', 'prader-labhart-willi syndrome']

NormMap sentence:  ['The', 'presence', 'at', 'the', 'birth', 'of', 'kyphoscoliosis', ',', 'associated', 'with', 'joint', 'hypermobility', 'and', 'the', 'absence', 'of', 'the', 'lingual', 'and', 'lower', 'lip', 'frenulum', ',', 'should', 'suggest', 'an', 'EDS', '.']
NormMap GARD ID:  GARD:0007674
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['spinal muscular atrophy']

NormMap sentence:  ['Despite', 'its', 'high', 'prevalence', ',', 'PCOS', 'and', 'its', 'accompanying', 'morbidities', 'are', 'likely', 'underdiagnosed', ',', 'averaging', '>', '2', 'years', 'and', '3', 'physicians', 'before', 'wom

DISEASE:  amebiasis
Exact Match sentence:  ['Therefore', ',', 'giardiasis', 'and', 'amebiasis', 'are', 'neglected', 'conditions', '.']
[4]
Exact Match GARD ID:  GARD:0000652
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['amebiasis', 'amoebic dysentery due to entamoeba histolytica', 'amoebiasis due to entamoeba histolytica', 'amebic dysentery', 'intestinal amebiasis']

DISEASE:  craniosynostosis
Exact Match sentence:  ['An', 'accurate', 'diagnosis', 'of', 'syndromic', 'craniosynostosis', '(', 'CS', ')', 'is', 'important', 'for', 'personalized', 'treatment', ',', 'surveillance', ',', 'and', 'genetic', 'counselling', '.']
[5]
Exact Match GARD ID:  GARD:0006209
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['craniosynostosis', 'craniostenosis']

NormMap sentence:  ['Only', '15', 'individuals', '(', '16', '%', ')', 'had', 'negative', 'genetic', 'analyses', '.']
NormMap GARD ID:  GARD:0000652
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['amebiasis', 'amoebic 

NormMap sentence:  ['In', '100', 'healthy', 'Roma', 'individuals', ',', 'none', 'related', 'to', 'the', 'analysed', 'families', ',', 'no', 'CYP21A2', 'mutations', 'were', 'detected', '.']
NormMap GARD ID:  GARD:0012074
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['oculo-auriculo-vertebral spectrum', 'oculo-auriculo-vertebral dysplasia']

NormMap sentence:  ['Almost', 'all', 'patients', 'had', 'the', 'severe', 'salt', '-', 'wasting', 'form', 'and', 'the', 'In2G', '/', 'In2', 'G', 'genotype', '.']
NormMap GARD ID:  GARD:0012521
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['multifocal fibrosclerosis', 'igg4-related disease', 'igg4-related systemic disease', 'igg4-syndrome', 'igg4-associated disease', 'igg4-related sclerosing disease', 'igg4-related systemic sclerosing disease', 'igg4-related autoimmune disease', 'igg4-positive multiorgan lymphoproliferative syndrome', 'hyper-igg4 disease', 'systemic igg4-related plasmacytic syndrome', 'systemic igg4-related scleros

NormMap sentence:  ['The', 'study', 'population', 'consisted', 'of', '4,489,044', 'live', '-', 'born', 'singletons', 'in', 'Denmark', 'during', '1978', '-', '2012', 'and', 'Sweden', 'during', '1987', '-', '2010', '.']
NormMap GARD ID:  GARD:0007830
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['tuberous sclerosis complex', 'tuberous sclerosis', 'bourneville syndrome']

DISEASE:  eclampsia
DISEASE:  eclampsia
NormMap sentence:  ['Maternal', 'HDP', 'was', 'categorized', 'into', 'chronic', 'hypertension', ',', 'gestational', 'hypertension', ',', 'and', 'pre', '-', 'eclampsia', ';', 'pre', '-', 'eclampsia', 'was', 'further', 'stratified', 'according', 'to', 'timing', '(', 'early', '-', 'onset', ',', 'late', '-', 'onset', ')', ',', 'or', 'severity', '(', 'moderate', ',', 'severe', ')', 'of', 'the', 'disease', '.']
NormMap GARD ID:  GARD:0010420
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['neurofibromatosis']
Exact Match sentence:  ['Maternal', 'HDP', 'was', 'categori

NormMap sentence:  ['Patients', 'and', 'methods', 'A', 'total', 'of', '160', 'conjunctival', 'pigmented', 'lesions', 'were', 'studied', '.']
NormMap GARD ID:  GARD:0001467
SEARCH TERM MATCHED TO GARD DICTIONARY. SEARCHING FOR:  ['congenital adrenal hyperplasia']



In [17]:
train_results

{'exact_tp': 665, 'exact_fn': 884, 'norm_tp': 897, 'norm_fn': 652}

In [18]:
val_results

{'exact_tp': 171, 'exact_fn': 190, 'norm_tp': 192, 'norm_fn': 169}

In [19]:
test_results

{'exact_tp': 76, 'exact_fn': 79, 'norm_tp': 87, 'norm_fn': 68}

In [20]:
def recall(tp,fn):
    return str(100*(tp/(tp+fn)))+'%'

In [21]:
recall(test_results['norm_tp'],test_results['norm_fn'])

'56.12903225806451%'

In [22]:
tp = train_results['norm_tp'] + val_results['norm_tp'] + val_results['norm_tp']
fn = train_results['norm_fn'] + val_results['norm_fn'] + val_results['norm_fn']
recall(tp,fn)

'56.40686922060766%'

### Saving the labels

In [ ]:
with open('epi_train_setV4.tsv', "w") as f:
    for i in range(len(mod_train_tokens)): #For sentence in list of sentences
        for j in range(len(mod_train_tokens[i])): #for token in sentence
            output = str(mod_train_tokens[i][j]) +'\t' +str(mod_train_labels[i][j])+'\n'
            f.write(output)
        f.write('\n')
f.close()

In [ ]:
with open('epi_val_setV4.tsv', "w") as f:
    for i in range(len(mod_val_tokens)): #For sentence in list of sentences
        for j in range(len(mod_val_tokens[i])): #for token in sentence
            output = str(mod_val_tokens[i][j]) +'\t' +str(mod_val_labels[i][j])+'\n'
            f.write(output)
        f.write('\n')
f.close()

In [ ]:
with open('epi_test_setV4.tsv', "w") as f:
    for i in range(len(mod_test_tokens)): #For sentence in list of sentences
        for j in range(len(mod_test_tokens[i])): #for token in sentence
            output = str(mod_test_tokens[i][j]) +'\t' +str(mod_test_labels[i][j])+'\n'
            f.write(output)
        f.write('\n')
f.close()